In [1]:
import requests
import json 
from bs4 import BeautifulSoup
from lxml import html
import re
import pandas as pd 
from tqdm import tqdm

# Retrieve information from each restaurant 

In [2]:
def extract_restaurant(url, header):
    response = requests.get(url, headers=header)
    
    # parsing 
    html_soup = BeautifulSoup(response.text, 'html.parser')
    tree = html.fromstring(response.content)
    
    #content 
    restaurant_page = {}
    restaurant_page['url'] = url

    # restaurant element
    # title 
    rest_title = ''
    try :
    # rest_title = html_soup.find_all('h1')[-1].text
        rest_title = tree.xpath('/html/body/div[1]/div/main/div/section[3]/section/section/div/div/div/h1')[0].text
        restaurant_page['title'] = rest_title
    except :
        pass 
    
    # extract restaurant type 
    rest_type = []
    for item in html_soup.find_all('span'): 
        try : 
            rest_type.append((item.find_all('a', href=True)[0].text))
        except : 
            pass 
    rest_type = '|'.join(rest_type[:-1])
    restaurant_page['rest_type'] = rest_type

    # price 
    rest_price = []
    
    for  item in html_soup.find_all('p'):
        if 'Rp' in item.text:
            rest_price.append(item.text)
    
    rest_price = '|'.join(rest_price)
    restaurant_page['rest_price'] = rest_price

    # facility 
    rest_facility = []
    
    for item in html_soup.find_all('p', {'class': re.compile(r'cunMUz')}):
        rest_facility.append(item.text)
    restaurant_page['rest_facility'] = '|'.join(rest_facility)

    # long lat 
    long_lat = [] 
    try : 
        for item in html_soup.find_all('a',href=True):
            if 'google' in item['href'] : 
                long_lat.append(item['href'])
            else : 
                pass 
    except :
        pass
        
    restaurant_page['rest_longlat'] = list(set(long_lat))

    
    # rating 
    rating = ''
    try : 
        rating = html_soup.find_all("div", {"class": "sc-1q7bklc-1 cILgox"})[0].text
        restaurant_page['rating'] = rating
    except : 
        pass 
    
    # total review 
    review = ''
    # review = tree.xpath('//div[normalize-space(text())="Reviews"]/preceding-sibling::div')

    try :
        review = tree.xpath('//*[@id="root"]/div/main/div/section[3]/section/section/div/div/div/section/div/div[2]/div[1]')
        review = review[0].text
        restaurant_page['review'] = review
    except : 
        restaurant_page['review'] = review
        
    return restaurant_page

In [21]:
# url = 'https://www.zomato.com/jakarta/monks-kelapa-gading'
url = 'https://www.zomato.com/jakarta/aroma-palopo-kelapa-gading/info'
my_headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0", 
          "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
              "Accept-Language":"en-US,en;q=0.5", 
             "Accept-Encoding":"gzip, deflate, br"}

rest_content = extract_restaurant(url, my_headers)

In [22]:
rest_content

{'url': 'https://www.zomato.com/jakarta/aroma-palopo-kelapa-gading/info',
 'title': 'Aroma Palopo',
 'rest_type': 'Makassar',
 'rest_price': 'Rp80.000 for two people (approx.)',
 'rest_facility': 'Home Delivery|Takeaway Available|Indoor Seating|Smoking Area|No Alcohol Available',
 'rest_longlat': ['https://www.google.com/maps/dir/?api=1&destination=-6.1714440000,106.8982310000'],
 'rating': '3.7',
 'review': '20'}

In [34]:
# for pickle file 

df = pd.read_pickle('data/restaurant_raw2_extra.pickle')
df = df.dropna()
df = df.rename(columns={'link':'rest_url'})
df = df.explode('rest_url')

In [24]:
# for csv input 

df = pd.read_csv('data/1_bodebek.csv')
df = df.dropna()
df = df.drop_duplicates()
df = df.rename(columns={'link':'rest_url'})
df = df.explode('rest_url')
# df2 = pd.read_csv('3_jak_rest_detail.csv')

In [10]:
# df.shape, df2.shape

In [11]:
# df[~df.rest_url.isin(df2['url'])]

In [13]:
# df['rest_url'].value_counts()

In [15]:
# df['rest_url'].value_counts()

In [11]:
# df3 = pd.merge(df,df2['url'],left_on='rest_url',right_on='url',how='left')

In [16]:
# df3.head()

In [17]:
# df3.shape

In [18]:
# df3.isnull().sum()

In [36]:
result = []
for url in tqdm(df.itertuples()):
    result.append(extract_restaurant(url.rest_url, my_headers))

2502it [38:56,  1.07it/s]


In [37]:
df_result = pd.DataFrame(result)

In [38]:
df_result.tail()

,url,title,rest_type,rest_price,rest_facility,rest_longlat,rating,review
2497,https://www.zomato.com/jakarta/coffee-san9a-be...,Coffee SAN9A,Cafe|Indonesian|Coffee|Tea,Rp80.000 for two people (approx.),Breakfast|Takeaway Available|Desserts and Bake...,[https://www.google.com/maps/dir/?api=1&destin...,4.1,157
2498,https://www.zomato.com/jakarta/kedai-abuya-bek...,Kedai Abuya,Indonesian,Rp100.000 for two people (approx.),Home Delivery|Takeaway Available|Indoor Seatin...,[https://www.google.com/maps/dir/?api=1&destin...,-,2
2499,https://www.zomato.com/jakarta/cafe-serabi-ban...,Cafe Serabi Banting & Coffee,Desserts|Indonesian,Rp80.000 for two people (approx.),Takeaway Available|Desserts and Bakes|Indoor S...,[https://www.google.com/maps/dir/?api=1&destin...,-,2
2500,https://www.zomato.com/jakarta/2kang-kopi-beka...,2kang Kopi,Coffee|Fusion,Rp100.000 for two people (approx.),Home Delivery|Takeaway Available|Indoor Seatin...,[https://www.google.com/maps/dir/?api=1&destin...,3.5,4
2501,https://www.zomato.com/jakarta/es-cow-bar-beka...,ES COW BAR,Beverages|Snacks|Indonesian,Rp100.000 for two people (approx.),Takeaway Available|Indoor Seating|No Alcohol A...,[https://www.google.com/maps/dir/?api=1&destin...,3.3,6


In [39]:
# df_result.to_csv('jakarta_restaurant_detail3.csv',index=False)
df_result.to_csv('data/2_bodebek_restaurant_detail3.csv',index=False)

In [ ]:
%%time
df['rest_info'] = df['rest_url'].apply(extract_restaurant, header = my_headers)

In [52]:
# set url 
url = 'https://www.zomato.com/jakarta/aroma-palopo-kelapa-gading/info'
response = requests.get(url, headers=my_headers)

# parsing 
html_soup = BeautifulSoup(response.text, 'html.parser')
tree = html.fromstring(response.content)

In [54]:
# testing area for xpath 
rating = tree.xpath('//div[normalize-space(text())="Reviews"]/preceding-sibling::div')

# tes = tree.xpath('/html/body/div[1]/div/main/div/section[4]/section/article/section/div[2]/a')

price = tree.xpath('//*[@id="root"]/div[2]/main/div/section[4]/section/section/article[1]/section[2]/p[2]')
# help(tes[0])
# print(tes[0].get_attribute("value"))

facility = tree.xpath('//*[@id="root"]/div[2]/main/div/section[4]/section/section/article[1]/section[2]/h3[4]')

In [55]:
# rating[0].text
for item in html_soup.find_all('p', {'class': re.compile(r'cunMUz')}):
    print (item.text)
# sc-1hez2tp-0 cunMUz

Home Delivery
Takeaway Available
Indoor Seating
Smoking Area
No Alcohol Available


In [16]:
main_url = 'https://www.zomato.com/jakarta'
response = requests.get(main_url, headers=my_headers)

# parsing 
html_soup = BeautifulSoup(response.text, 'html.parser')
tree = html.fromstring(response.content)

In [17]:
# extract 

rest_jakarta = tree.xpath('/html/body/div[1]/div/div[4]/div[4]/div[2]/div')

In [42]:
for item in rest_jakarta[0][0]:
#     print(item.tag)
    for subitem in item:
        print(subitem.tag)

div


In [43]:
print(rest_jakarta)

[<Element div at 0x1e07c79e0e8>]
